In [ ]:

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Mar 20 23:49:14 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   29C    P0    52W / 400W |  17953MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#install aimodelshare library
! pip install aimodelshare --upgrade
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as functional
import matplotlib.pyplot as plt
from transformers import BertForSequenceClassification, AdamW, BertConfig
import gc
from transformers import BertModel
from sklearn.metrics import roc_auc_score,f1_score
import time
import datetime

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
torch.manual_seed(0)
use_cuda = torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')
if use_cuda:
    torch.cuda.manual_seed(0)
    
print("Using GPU: {}".format(use_cuda))

Using GPU: True


In [ ]:
# Read the rumor dataset to train the model parameters
df = pd.read_csv('/content/drive/My Drive/Misinformation_Project/rumor_dataset.csv') 
df["Veracity"] = df.Veracity.map({"F":0, "T":1, "U":0}) 
df.replace([np.inf, -np.inf], 2) 
df.fillna(2, inplace=True)
df.head()

,Index,Veracity,Text,Sentiment
0,3,0.0,The lie that coronavirus came from a bat or a ...,3
1,4,0.0,The health experts had predicted the virus cou...,3
2,8,0.0,The Centers for Disease Control and Prevention...,3
3,10,0.0,Warm weather will kill coronavirus. U.S. Presi...,2
4,15,0.0,Using a hair dryer to breathe in hot air can c...,2


In [ ]:
# Split into traning and testing sets
from sklearn.model_selection import train_test_split 

train, test = train_test_split(df, test_size=0.2)

In [ ]:
# Import libraries for the model training
import tensorflow as tf
import re, string, unicodedata
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPool1D, Dropout, Input
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

#BERT
import transformers
from transformers import TFBertModel
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tqdm.notebook import tqdm
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')

In [ ]:
def bert_encode(data, maximum_length) :
    input_ids = []
    attention_masks = []

    for text in data:
        encoded = tokenizer.encode_plus(
            text, 
            add_special_tokens=True,
            max_length=maximum_length,
            pad_to_max_length=True,

            return_attention_mask=True,
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
        
    return np.array(input_ids),np.array(attention_masks)

In [ ]:
def create_model(bert_model):
    
    input_ids = tf.keras.Input(shape=(310,),dtype='int32')
    attention_masks = tf.keras.Input(shape=(310,),dtype='int32')

    output = bert_model([input_ids,attention_masks])
    output = output[1]
    output = tf.keras.layers.Dense(64,activation='relu')(output)
    output = tf.keras.layers.Dropout(0.2)(output)
    output = tf.keras.layers.Dense(1,activation='sigmoid')(output)
    
    model = tf.keras.models.Model(inputs = [input_ids,attention_masks],outputs = output)
    model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    return model
     

In [ ]:
#load pre-trained uncased BERT
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
model_BERT_loss1e5_epoch5 = create_model(bert_model)
model_BERT_loss1e5_epoch5.summary

/usr/local/lib/python3.9/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


<bound method Model.summary of <keras.engine.functional.Functional object at 0x7f6acdf53100>>

In [ ]:
train_input_ids, train_attention_masks = bert_encode(train["Text"], 310)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2346: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
history = model_BERT_loss1e5_epoch5.fit(
    [train_input_ids, train_attention_masks],
    train["Veracity"],
    epochs=5,
    batch_size=16
)

Epoch 1/5
308/308 [==============================] - 59s 140ms/step - loss: 0.4484 - accuracy: 0.7886
Epoch 2/5
308/308 [==============================] - 43s 140ms/step - loss: 0.3333 - accuracy: 0.8514
Epoch 3/5
308/308 [==============================] - 43s 140ms/step - loss: 0.2463 - accuracy: 0.8902
Epoch 4/5
308/308 [==============================] - 43s 139ms/step - loss: 0.1665 - accuracy: 0.9295
Epoch 5/5
308/308 [==============================] - 43s 139ms/step - loss: 0.0975 - accuracy: 0.9624


In [ ]:
test_input_ids, test_attention_masks = bert_encode(test["Text"], 310)

In [ ]:
predictions = model_BERT_loss1e5_epoch5.predict((test_input_ids, test_attention_masks))

39/39 [==============================] - 6s 89ms/step


In [ ]:
prediction_labels = [1 if x > 0.5 else 0 for x in predictions]
prediction_labels_df = pd.DataFrame({"labels":prediction_labels,"tweet":test["Text"]})

In [ ]:
prediction_labels_df.to_csv("/content/drive/MyDrive/Colab Notebooks/bert_3.csv")

In [ ]:
!pip install -U tf2onnx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tf2onnx
import onnx

model_proto, external_tensor_storage = tf2onnx.convert.from_keras(model_BERT_loss1e5_epoch5)
onnx.save(model_proto, "/content/drive/MyDrive/Colab Notebooks/final_model_3.onnx")
model_BERT_loss1e5_epoch5.save_weights("/content/drive/MyDrive/Colab Notebooks/bert_weights_3")
    

In [ ]:
# read our own intervention-related Twitter data
df_interventions = pd.read_csv('/content/drive/My Drive/Misinformation_Project/21-22 two year updated dataset/individualinf_repsonseeff_30K_interventions_21_22.csv') 

In [ ]:
df_interventions["Processed Tweets"].isnull().sum()

1

In [ ]:
df_interventions["Processed Tweets"].fillna(" ", inplace=True)

In [ ]:
df_interventions["Processed Tweets"].isnull().sum()

0

In [ ]:
interventions_input_ids, interventions_attention_masks = bert_encode(df_interventions["Processed Tweets"], 310)

In [ ]:
interventions_predictions = model_BERT_loss1e5_epoch5.predict((interventions_input_ids, interventions_attention_masks))

1114/1114 [==============================] - 101s 91ms/step


In [ ]:
df_interventions["labels"] = [1 if x > 0.5 else 0 for x in interventions_predictions]
df_interventions["Classify"]=df_interventions.labels.map({0:"M", 1:"C"}) 

In [ ]:
df_interventions.head()

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,User,Date Created,Tweet,Likes,Followers,Friends,...,Polarity,Subjectivity,Like Effect,Reply Effect,Retweet Effect,Individual Influence,Response Effect,Subjective Level,labels,Classify
0,0,0,0,0,HSans,2022-12-30 23:42:46+00:00,"Here in #Québec, #Canada we've had this rule i...",0,2335,4527,...,0.500000,0.500000,0,0,0,3,0.0,3,0,M
1,1,1,1,1,Randi_RLB,2022-12-30 23:11:27+00:00,"@DrAGrace_cyhtt ""Alberta's education minister ...",0,1531,2204,...,-0.117262,0.577381,0,0,0,3,0.0,3,0,M
2,2,2,2,2,Golden_Pup,2022-12-30 22:51:45+00:00,November 18\n\nJust three days after defying a...,196,39775,584,...,0.000000,0.000000,3,1,2,4,2.0,1,1,C
3,3,3,3,3,EnigmaMachine8,2022-12-30 19:14:24+00:00,@shogun17761 @Katheri58113861 @kirkJK04 @jeanp...,0,2957,3137,...,0.000000,1.000000,0,0,0,3,0.0,4,1,C
4,4,4,4,4,Cognisant2000,2022-12-30 18:59:47+00:00,@GMMalliet Agree. We always wear a mask in enc...,0,2037,1713,...,0.000000,0.000000,0,0,0,3,0.0,1,0,M


In [ ]:
# Function to determine sign of impact based on polarity of sentiment 
# We assume negative opinion on vaccine has a negative impact
def signofimpact(score):
    if score > 0:
        return 1
    else:
        return -1

# Function to check if the tweet contains hashtags that are hot topics
# Have a hashtag under a hot topic will increase visibility of the tweet
def ifhottopic(hashtags):
    match = ['COVID' , 'Toronto','Ottawa','cdnpoli','yyc','Vancouver','covid','Quebec','onpoli','Canada']
    if any(x in hashtags for x in match):
        return 1
    else:
        return 0

In [ ]:
df_interventions["Sign of Impact"] = df_interventions["Polarity"].apply(signofimpact)
df_interventions["Impact Value"] = df_interventions["Individual Influence"]+df_interventions["Response Effect"]+df_interventions["Subjectivity"]+df_interventions["Hashtags"].astype(str).apply(ifhottopic)
df_interventions["Overall Impact"] = df_interventions["Sign of Impact"]*df_interventions["Impact Value"]
df_interventions.head()

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,User,Date Created,Tweet,Likes,Followers,Friends,...,Reply Effect,Retweet Effect,Individual Influence,Response Effect,Subjective Level,labels,Classify,Sign of Impact,Impact Value,Overall Impact
0,0,0,0,0,HSans,2022-12-30 23:42:46+00:00,"Here in #Québec, #Canada we've had this rule i...",0,2335,4527,...,0,0,3,0.0,3,0,M,1,4.500000,4.500000
1,1,1,1,1,Randi_RLB,2022-12-30 23:11:27+00:00,"@DrAGrace_cyhtt ""Alberta's education minister ...",0,1531,2204,...,0,0,3,0.0,3,0,M,-1,3.577381,-3.577381
2,2,2,2,2,Golden_Pup,2022-12-30 22:51:45+00:00,November 18\n\nJust three days after defying a...,196,39775,584,...,1,2,4,2.0,1,1,C,-1,6.000000,-6.000000
3,3,3,3,3,EnigmaMachine8,2022-12-30 19:14:24+00:00,@shogun17761 @Katheri58113861 @kirkJK04 @jeanp...,0,2957,3137,...,0,0,3,0.0,4,1,C,-1,4.000000,-4.000000
4,4,4,4,4,Cognisant2000,2022-12-30 18:59:47+00:00,@GMMalliet Agree. We always wear a mask in enc...,0,2037,1713,...,0,0,3,0.0,1,0,M,-1,3.000000,-3.000000


In [ ]:
df_interventions.to_csv('/content/drive/My Drive/Misinformation_Project/Bert_classified_impact_score_30K_interventions_21_22.csv')